In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px

In [ ]:
df_tesla=pd.read_csv("Tesla Stock Price (2010 to 2023).csv")


In [ ]:
df_tesla.head()

In [ ]:
df_tesla.shape

In [ ]:
df_tesla['Date']=pd.to_datetime(df_tesla.Date)


In [ ]:
df_tesla=df_tesla.set_index('Date',drop=True)

In [ ]:
df_tesla.head()

In [ ]:
df_tesla["Open"].plot(figsize=(12,5))
fig=px.line(df_tesla,x=df_tesla.index,y="Open")
fig.show()

ROLLING

In [ ]:
#simple moving average
df_tesla['open:10 days rolling']=df_tesla["Open"].rolling(window=10,min_periods=1).mean()

In [ ]:
df_tesla[['Open','open:10 days rolling']].plot(figsize=(15,5))

In [ ]:
df_tesla['open:30 days rolling']=df_tesla["Open"].rolling(window=30,min_periods=1).mean()
df_tesla['open:50 days rolling']=df_tesla["Open"].rolling(window=50,min_periods=1).mean()
df_tesla['open:100 days rolling']=df_tesla["Open"].rolling(window=100,min_periods=1).mean()

In [ ]:
fig=px.line(df_tesla,x=df_tesla.index,y=['Open','open:10 days rolling','open:30 days rolling','open:50 days rolling'],width=1100,height=700)
fig.show()

EXPANDING

In [ ]:
#Cumulative moving average
df_tesla["Open"].expanding().mean().plot(figsize=(10,5))

EXPONENTIAL MOVING AVERAGE

In [ ]:
df_tesla['EMA_0.1']=df_tesla["Open"].ewm(alpha=0.1,adjust=False).mean()

In [ ]:
fig=px.line(df_tesla,x=df_tesla.index,y=['Open','EMA_0.1'],width=1100,height=700)
fig.show()

In [ ]:
df_tesla['EMA_0.3']=df_tesla["Open"].ewm(alpha=0.3,adjust=False).mean()

In [ ]:
fig=px.line(df_tesla,x=df_tesla.index,y=['Open','EMA_0.1','EMA_0.3'],width=1100,height=700)
fig.show()

In [ ]:
#EWMA
df_tesla['EMA_5days']=df_tesla["Open"].ewm(span=5).mean()

In [ ]:
fig=px.line(df_tesla,x=df_tesla.index,y=['Open','EMA_5days'],width=1100,height=700)
fig.show()

In [ ]:
fig=px.line(df_tesla,x=df_tesla.index,y=['Open','EMA_0.3','EMA_5days'],width=1000,height=700)
fig.show()

In [ ]:
import statsmodels as sm
import statsmodels.robust

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
import statsmodels.api as sms


In [ ]:
adft = adfuller(df_tesla.Open)

In [ ]:
pd.Series(adft[0:4],index=['test stats','p value','lag','data points'])

In [ ]:
def adf_test(series):
    result=adfuller(series)
    print('ADF Statistics: {}'.format(result[0]))
    print('p- value: {}'.format(result[1]))
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")
    

In [ ]:
df_tesla['Open_Diff']=df_tesla["Open"]-df_tesla["Open"].shift(1)

In [ ]:
df_tesla.head()

In [ ]:
adf_test(df_tesla["Open_Diff"].dropna())

In [ ]:
sns.boxplot(df_tesla.Open)

In [ ]:
# time series decomposition 
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df_tesla[['Open']],period=12)

In [ ]:
result.plot() 

In [ ]:
result.seasonal

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
acf = plot_acf(df_tesla["Open"])


In [ ]:
# ACF and PACF 
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
plot_acf(df_tesla.Open)
plot_pacf(df_tesla.Open)

In [ ]:
df_tesla.Open_Diff

In [ ]:
df_open=df_tesla['Open_Diff'].dropna()

In [ ]:
df_open

In [ ]:
plt.plot(df_open)

In [ ]:
adf = adfuller(df_open)

In [ ]:
adf[0:4]

In [ ]:
train_data = df_open[0:-650]
test_data = df_open[-650:]


In [ ]:
train_data

In [ ]:
test_data

In [ ]:
import statsmodels.api as sms
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

In [ ]:
history=[x for x in train_data]

In [ ]:
model=ARIMA(history,order=(1,1,1))

In [ ]:
model=model.fit()

In [ ]:
model.summary()

In [ ]:
model.forecast()

In [ ]:
mean_squared_error([test_data[0]],model.forecast())

In [ ]:
def train_arima_model(X, y, arima_order):
    # prepare training dataset
    # make predictions list
    history = [x for x in X]
    predictions = list()
    for t in range(len(y)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(y[t])
    # calculate out of sample error
    rmse = np.sqrt(mean_squared_error(y, predictions))
    return rmse

In [ ]:
def evaluate_models (dataset, test, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None 
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    rmse = train_arima_model(dataset, test, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print ('ARIMA%s RMSE=%.3f' % (order, rmse)) 
                except:
                    continue
    print( 'Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))

In [ ]:

# evaluate parameters
import warnings
warnings.filterwarnings('ignore')
p_values = range(0, 3)
d_values = range(0, 3)
q_values = range(0, 3)
evaluate_models(train_data, test_data, p_values, d_values, q_values)

In [ ]:
history = [x for x in train_data]
predictions = list()
conf_list = list()
for t in range(len(test_data)):
    model = ARIMA(history,order=(2,0,0))
    model_fit = model.fit()
    fc = model_fit.forecast(alpha = 0.05)
    predictions.append(fc)
    history.append(test_data[t])
print('RMSE of ARIMA Model:', np.sqrt(mean_squared_error(test_data, predictions)))

In [ ]:
plt.figure(figsize=(18,8))
plt.grid(True)
plt.plot(range(len(test_data)),test_data, label = 'True Test Close Value', linewidth = 5)
plt.plot(range(len(predictions)), predictions, label = 'Predictions on test data', linewidth = 5)
plt.xticks(fontsize = 15)
plt.xticks(fontsize = 15)
plt.legend(fontsize = 20, shadow=True,facecolor='lightpink',edgecolor = 'k')
plt.show()

In [ ]:
fc_series = pd.Series(predictions, index=test_data.index)

In [ ]:
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train_data, label='Training', color = 'blue')
plt.plot(test_data, label='Test', color = 'green', linewidth = 3)
plt.plot(fc_series, label='Forecast', color = 'red')
plt.title('Forecast vs Actuals on test data')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_predict
fig = plt.figure(figsize=(18,8))
ax1 = fig.add_subplot(111)
plot_predict(result=model_fit,start=1, end=len(df_open)+60, ax = ax1)
plt.grid("both")
plt.legend(['Forecast','Open','95% confidence interval'],fontsize = 20, shadow=True,facecolor='lightblue',edgecolor = 'k')
plt.show()

In [ ]:
# evaluate parameters
import warnings
warnings.filterwarnings('ignore')
history = [x for x in train_data]
predictions = list()
conf_list = list()
for t in range(len(test_data)):
    model = sms.tsa.statespace.SARIMAX(history, order = (0,1,0), seasonal_order = (1,1,1,3))
    model_fit = model.fit()
    fc = model_fit.forecast()
    predictions.append(fc)
    history.append(test_data[t])
print('RMSE of SARIMA Model:', np.sqrt(mean_squared_error(test_data, predictions)))

In [ ]:
plt.figure(figsize=(18,8))
plt.title('Forecast vs Actual', fontsize = 25)
plt.plot(range(60), predictions, label = 'Predictions', linewidth = 4)
plt.plot(range(60), test_data, label = 'Close', linewidth = 4)
plt.legend(fontsize = 25, shadow=True,facecolor='lightpink',edgecolor = 'k')